# Entregas Semanales DP

## Foto Datos DP

In [1]:
from configparser import ConfigParser

from snowflake.connector import connect #type: ignore

from snowflake.snowpark import Session

from pandas import DataFrame, concat, Series

from datetime import date

import snowflake.snowpark.functions as fn

config = ConfigParser()
config.read('config.ini')

connection_parameters: dict[str, int | str] = {
    'user' : config.get('SNOWFLAKE', 'USER'),
    'password' : config.get('SNOWFLAKE', 'PASSWORD'),
    'account' : config.get('SNOWFLAKE', 'ACCOUNT'),
    'database' : config.get('SNOWFLAKE', 'DATABASE'),
    'warehouse' : config.get('SNOWFLAKE', 'WAREHOUSE'),
    'schema' : config.get('SNOWFLAKE', 'SCHEMA'),
    'role' : config.get('SNOWFLAKE', 'ROLE'),
}

session = Session.builder.configs(connection_parameters).create()

conn = connect(
    user = config.get('SNOWFLAKE', 'USER'),
    password = config.get('SNOWFLAKE', 'PASSWORD'),
    account = config.get('SNOWFLAKE', 'ACCOUNT'),
    database = config.get('SNOWFLAKE', 'DATABASE'),
    warehouse = config.get('SNOWFLAKE', 'WAREHOUSE'),
    schema = config.get('SNOWFLAKE', 'SCHEMA'),
    role = config.get('SNOWFLAKE', 'ROLE'),
)

SEMANA_ACTUAL = 45
MES_ACTUAL = 11
ANIO_ACTUAL = 2023

SEMANA_PREVIA = 44
ANIO_SEMANA_PREVIA = 2023

MES_PREVIO = 10
ANIO_MES_PREVIO = 2023

ANIO_PREVIO = 2022

### Descargas App

In [2]:
from DP_instalaciones_aplicacion import get_app_installs_DP
from snowflake.snowpark.functions import count
from general_generar_reporte import generate_report

tiempo = session.table('WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME')

instalaciones = (
    get_app_installs_DP(session)
    .join(tiempo, on = 'FECHA')
)
instalaciones = generate_report(instalaciones, count, '*', ANIO_ACTUAL, SEMANA_ACTUAL, ANIO_SEMANA_PREVIA, SEMANA_PREVIA, MES_ACTUAL, ANIO_MES_PREVIO, MES_PREVIO, ANIO_PREVIO, 'INSTALACIONES').to_pandas()

instalaciones.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO,DATO
0,136646,127620,0.070726,264266,300085,-0.119363,1976277,3115509,-0.365665,INSTALACIONES


### Primeras compras

In [3]:
from DP_transacciones_total import get_transacciones_dp_total
from general_primeras_compras import get_primeras_compras
from snowflake.snowpark.functions import count_distinct

transacciones = get_transacciones_dp_total(session)

primeras_compras = (
    get_primeras_compras(transacciones)
    .join(tiempo, on = 'FECHA')
)

primeras_compras = generate_report(primeras_compras, count_distinct, 'EMAIL', ANIO_ACTUAL, SEMANA_ACTUAL, ANIO_SEMANA_PREVIA, SEMANA_PREVIA, MES_ACTUAL, ANIO_MES_PREVIO, MES_PREVIO, ANIO_PREVIO, 'PRIMERAS_COMPRAS').to_pandas()

primeras_compras.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO,DATO
0,37424,29900,0.251639,67324,100871,-0.332573,1053116,1116817,-0.057038,PRIMERAS_COMPRAS


### Registros Totales

In [4]:
from DP_registros import get_registros_dp

registros = (
    get_registros_dp(session)
    .join(tiempo, on = 'FECHA')
)

registros_totales = generate_report(registros, count_distinct, 'EMAIL', ANIO_ACTUAL, SEMANA_ACTUAL, ANIO_SEMANA_PREVIA, SEMANA_PREVIA, MES_ACTUAL, ANIO_MES_PREVIO, MES_PREVIO, ANIO_PREVIO, 'REGISTROS_TOTALES').to_pandas()

registros_totales.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO,DATO
0,17167,15111,0.136060,32269,72881,-0.557237,340723,0,0.000000,REGISTROS_TOTALES


### Registros con compra

In [5]:
registros_con_compra = conn.cursor().execute(f"""
    WITH

    TRANSACCIONES_OLO AS (
        SELECT DISTINCT
            lower(EMAIL) AS EMAIL,
            to_date(MXP.ORDERDATE) AS FECHA
        FROM
        "SEGMENT_EVENTS"."DOMINOS_OLO"."MXPOWERSALESLOG" MXP
        INNER JOIN 
            "SEGMENT_EVENTS"."DOMINOS_OLO"."DPMSALES_FULL" OLO
        ON 
            OLO.ORDER_NUMBER = MXP.ORDERNUMBER AND OLO.LOCATION_CODE = MXP.STORENUMBER 
        AND 
            TO_CHAR(OLO.ORDER_DATE,'YYYY-MM-DD') = MXP.ORDERDATE
        WHERE 
            OLO.ORDER_STATUS_CODE = 4
        AND 
            OLO.LOCATION_CODE NOT IN ('13001' , '13006', '13021', '11000')
        AND 
            UPPER(OLO.SOURCE_CODE) IN ('ANDROID' , 'DESKTOP', 'IOS', 'MOBILE', 'WEB', 'ANDROID2', 'DESKTOP2', 'IOSAPP', 'MOBILE2', 'WHATSAPP')
        GROUP BY
            EMAIL,
            FECHA
    ),

    TRANSACCIONES_CLOUD AS (
        SELECT DISTINCT 
            lower(A.EMAIL) AS EMAIL,
            to_date(SUBSTRING(A.STOREORDERID,1,10)) AS FECHA
        FROM 
            "SEGMENT_EVENTS"."DOMINOS_GOLO"."VENTA_CLOUD" A
        WHERE 
            A.STOREID NOT LIKE '9%'
        AND 
            A.SOURCEORGANIZATIONURI IN ('order.dominos.com','resp-order.dominos.com','iphone.dominos.mx','android.dominos.mx') 
        AND 
            A.SOURCEORGANIZATIONURI IS NOT NULL
        GROUP BY
            EMAIL,
            FECHA 
    ),

    TRANSACCIONES_TOTALES AS (
        SELECT
            *
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        INNER JOIN
        (
            SELECT * FROM TRANSACCIONES_OLO
            UNION ALL
            SELECT * FROM TRANSACCIONES_CLOUD
        )
        USING(
            FECHA
        )
    ),

    CON_COMPRA_ACTUAL AS (
        SELECT DISTINCT
            EMAIL
        FROM
            TRANSACCIONES_TOTALES
        WHERE
            ANIO_ALSEA < {ANIO_ACTUAL}    
        OR
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
    ), 

    SEMANA_ACTUAL AS (
        SELECT
            count(DISTINCT EMAIL) AS SEMANA_ACTUAL
        FROM
            SEGMENT_EVENTS.DOMINOS_UNIFIED.SIGNUP_SUCCESS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SENT_AT) = FECHA
        INNER JOIN
            CON_COMPRA_ACTUAL
        USING(
            EMAIL
        )
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA = {SEMANA_ACTUAL}
    ),

    CON_COMPRA_HASTA_SEMANA_PREVIA AS (
        SELECT DISTINCT
            EMAIL
        FROM
            TRANSACCIONES_TOTALES
        WHERE
            ANIO_ALSEA < {ANIO_SEMANA_PREVIA}    
        OR
            ANIO_ALSEA = {ANIO_SEMANA_PREVIA}
        AND
            SEM_ALSEA <= {SEMANA_PREVIA}
    ),

    SEMANA_PREVIA AS (
        SELECT
            count(DISTINCT EMAIL) AS SEMANA_PREVIA
        FROM
            SEGMENT_EVENTS.DOMINOS_UNIFIED.SIGNUP_SUCCESS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SENT_AT) = FECHA
        INNER JOIN
            CON_COMPRA_HASTA_SEMANA_PREVIA
        USING(
            EMAIL
        )
        WHERE
            ANIO_ALSEA = {ANIO_SEMANA_PREVIA}
        AND
            SEM_ALSEA = {SEMANA_PREVIA}
    ),

    SEMANAL AS (
        SELECT
            *,
            (SEMANA_ACTUAL - SEMANA_PREVIA) / SEMANA_PREVIA AS PERCENT_V_SEMANA_PREVIA
        FROM
            SEMANA_ACTUAL
        JOIN
            SEMANA_PREVIA
    ),

    MES_ACTUAL AS (
        SELECT
            count(DISTINCT EMAIL) AS MES_ACTUAL
        FROM
            SEGMENT_EVENTS.DOMINOS_UNIFIED.SIGNUP_SUCCESS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SENT_AT) = FECHA
        INNER JOIN
            CON_COMPRA_ACTUAL
        USING(
            EMAIL
        )
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            MES_ALSEA = {MES_ACTUAL}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
    ),

    CON_COMPRA_HASTA_MES_PREVIO AS (
        SELECT DISTINCT
            EMAIL
        FROM
            TRANSACCIONES_TOTALES
        WHERE
            ANIO_ALSEA < {ANIO_MES_PREVIO}    
        OR
            ANIO_ALSEA = {ANIO_MES_PREVIO}
        AND
            MES_ALSEA <= {MES_PREVIO}
    ),

    MES_PREVIO AS (
        SELECT
            count(DISTINCT EMAIL) AS MES_PREVIO
        FROM
            SEGMENT_EVENTS.DOMINOS_UNIFIED.SIGNUP_SUCCESS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SENT_AT) = FECHA
        INNER JOIN
            CON_COMPRA_HASTA_MES_PREVIO
        USING(
            EMAIL
        )
        WHERE
            ANIO_ALSEA = {ANIO_MES_PREVIO}
        AND
            MES_ALSEA = {MES_PREVIO}
    ),

    MENSUAL AS (
        SELECT
            *,
            (MES_ACTUAL - MES_PREVIO) / MES_PREVIO AS PERCENT_V_MES_PREVIO
        FROM
            MES_ACTUAL
        JOIN
            MES_PREVIO
    ),

    ANIO_ACTUAL AS (
        SELECT
            count(DISTINCT EMAIL) AS ANIO_ACTUAL
        FROM
            SEGMENT_EVENTS.DOMINOS_UNIFIED.SIGNUP_SUCCESS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SENT_AT) = FECHA
        INNER JOIN
            CON_COMPRA_ACTUAL
        USING(
            EMAIL
        )
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
    ),

    CON_COMPRA_HASTA_ANIO_PREVIO AS (
        SELECT DISTINCT
            EMAIL
        FROM
            TRANSACCIONES_TOTALES
        WHERE
            ANIO_ALSEA <= {ANIO_PREVIO}
    ),

    ANIO_PREVIO AS (
        SELECT
            count(DISTINCT EMAIL) AS ANIO_PREVIO
        FROM
            SEGMENT_EVENTS.DOMINOS_UNIFIED.SIGNUP_SUCCESS 
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SENT_AT) = FECHA
        INNER JOIN
            CON_COMPRA_HASTA_ANIO_PREVIO
        USING(
            EMAIL
        )
        WHERE
            ANIO_ALSEA = {ANIO_PREVIO}
        AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
    ),

    ANUAL AS (
        SELECT
            *,
            CASE
                WHEN ANIO_PREVIO <> 0 THEN (ANIO_ACTUAL - ANIO_PREVIO) / ANIO_PREVIO
                ELSE 0
            END AS PERCENT_V_ANIO_PREVIO
        FROM
            ANIO_ACTUAL
        JOIN
            ANIO_PREVIO
    )

    SELECT
        *
    FROM
        SEMANAL
    JOIN
        MENSUAL
    JOIN
        ANUAL
    ;
""").fetch_pandas_all()

registros_con_compra['DATO'] = 'REGISTROS_CON_COMPRA'

registros_con_compra.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO,DATO
0,10708,8669,0.235206,19695,40023,-0.507908,191939,0,0.000000,REGISTROS_CON_COMPRA


### Registros sin compra

In [6]:
semana_actual = registros_totales['SEMANA_ACTUAL'].iloc[0] - registros_con_compra['SEMANA_ACTUAL'].iloc[0]
semana_previa = registros_totales['SEMANA_PREVIA'].iloc[0] - registros_con_compra['SEMANA_PREVIA'].iloc[0]
percent_v_semana_previa = (semana_actual - semana_previa) / semana_previa

mes_actual = registros_totales['MES_ACTUAL'].iloc[0] - registros_con_compra['MES_ACTUAL'].iloc[0]
mes_previo = registros_totales['MES_PREVIO'].iloc[0] - registros_con_compra['MES_PREVIO'].iloc[0]
percent_v_mes_previo = (mes_actual - mes_previo) / mes_previo

anio_actual = registros_totales['ANIO_ACTUAL'].iloc[0] - registros_con_compra['ANIO_ACTUAL'].iloc[0]
anio_previo = registros_totales['ANIO_PREVIO'].iloc[0] - registros_con_compra['ANIO_PREVIO'].iloc[0]
if anio_previo != 0:
    percent_v_anio_previo = (anio_actual - anio_previo) / anio_previo 
else:
    percent_v_anio_previo = 0

data = {
    'SEMANA_ACTUAL' : [semana_actual],
    'SEMANA_PREVIA' : [semana_previa],
    'PERCENT_V_SEMANA_PREVIA' : [percent_v_semana_previa],
    'MES_ACTUAL' : [mes_actual],
    'MES_PREVIO' : [mes_previo],
    'PERCENT_V_MES_PREVIO' : [percent_v_mes_previo],
    'ANIO_ACTUAL' : [anio_actual],
    'ANIO_PREVIO' : [anio_previo],
    'PERCENT_V_ANIO_PREVIO' : [percent_v_anio_previo],
    'DATO' : 'REGISTROS_SIN_COMPRA'
}

registros_sin_compra = DataFrame(data)

registros_sin_compra.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO,DATO
0,6459,6442,0.002639,12574,32858,-0.617323,148784,0,0,REGISTROS_SIN_COMPRA


### Activos

In [7]:
activos = conn.cursor().execute(f"""
    WITH

    PERIODO_ACTUAL AS (
        SELECT
            to_date(max(FECHA)) AS FECHA_FIN,
            FECHA_FIN - 180 AS FECHA_INICIO
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA = {SEMANA_ACTUAL}
    ),

    SEMANA_PREVIA AS (
        SELECT
            to_date(max(FECHA)) AS FECHA_FIN,
            FECHA_FIN - 180 AS FECHA_INICIO
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA = {SEMANA_PREVIA}
    ),

    MES_PREVIO AS (
        SELECT
            to_date(max(FECHA)) AS FECHA_FIN,
            FECHA_FIN - 180 AS FECHA_INICIO
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            MES_ALSEA = {MES_PREVIO}
    ),

    ANIO_PREVIO AS (
        SELECT
            to_date(max(FECHA)) AS FECHA_FIN,
            FECHA_FIN - 180 AS FECHA_INICIO
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
            ANIO_ALSEA = {ANIO_PREVIO}
        AND
            SEM_ALSEA = {SEMANA_ACTUAL}
    ),

    TRANSACCIONES_OLO AS (
        SELECT DISTINCT
            lower(MXP.EMAIL) AS EMAIL,
            ORDER_DATE AS FECHA,
            TO_CHAR(ORDER_DATE,'YYYY-MM-DD')||LOCATION_CODE||OLO.ORDER_NUMBER AS ORDER_ID,
            OLO.ORDERFINALPRICE / 1.16 AS ORDER_AMOUNT
        FROM
        "SEGMENT_EVENTS"."DOMINOS_OLO"."MXPOWERSALESLOG" MXP
        INNER JOIN 
            "SEGMENT_EVENTS"."DOMINOS_OLO"."DPMSALES_FULL" OLO
        ON 
            OLO.ORDER_NUMBER = MXP.ORDERNUMBER AND OLO.LOCATION_CODE = MXP.STORENUMBER 
        AND 
            TO_CHAR(OLO.ORDER_DATE,'YYYY-MM-DD') = MXP.ORDERDATE
        WHERE 
            OLO.ORDER_STATUS_CODE = 4
        AND 
            OLO.LOCATION_CODE NOT IN ('13001' , '13006', '13021', '11000')
        AND 
            UPPER(OLO.SOURCE_CODE) IN (
                'ANDROID', 
                'DESKTOP', 
                'IOS', 
                'MOBILE', 
                'WEB', 
                'ANDROID2', 
                'DESKTOP2', 
                'IOSAPP', 
                'MOBILE2', 
                'WHATSAPP'
            )
    ),

    TRANSACCIONES_CLOUD AS (
        SELECT DISTINCT 
            lower(A.EMAIL) AS EMAIL,
            to_date(SUBSTRING(A.STOREORDERID,1,10)) AS FECHA,
            to_char(FECHA) || A.STOREID ||A.StoreOrderID AS ORDER_ID,
            PAYMENTSAMOUNT / 1.16 AS ORDER_AMOUNT
        FROM 
            "SEGMENT_EVENTS"."DOMINOS_GOLO"."VENTA_CLOUD" A
        WHERE 
            A.STOREID NOT LIKE '9%'
        AND 
            A.SOURCEORGANIZATIONURI IN (
                'order.dominos.com', 
                'resp-order.dominos.com', 
                'iphone.dominos.mx', 
                'android.dominos.mx'
            ) 
        AND 
            A.SOURCEORGANIZATIONURI IS NOT NULL
    ),

    TRANSACCIONES_OLO_Y_CLOUD AS (
        SELECT * FROM TRANSACCIONES_OLO
        UNION ALL
        SELECT * FROM TRANSACCIONES_CLOUD
    ),

    ACTIVOS_PERIODO_ACTUAL AS (
        SELECT
            count(DISTINCT EMAIL) AS CLIENTES
        FROM
            TRANSACCIONES_OLO_Y_CLOUD
        INNER JOIN
            PERIODO_ACTUAL
        ON
            FECHA BETWEEN FECHA_INICIO AND FECHA_FIN
    ),

    ACTIVOS_SEMANA_PREVIA AS (
        SELECT
            count(DISTINCT EMAIL) AS CLIENTES
        FROM
            TRANSACCIONES_OLO_Y_CLOUD
        INNER JOIN
            SEMANA_PREVIA
        ON
            FECHA BETWEEN FECHA_INICIO AND FECHA_FIN
    ),

    ACTIVOS_MES_PREVIO AS (
        SELECT
            count(DISTINCT EMAIL) AS CLIENTES
        FROM
            TRANSACCIONES_OLO_Y_CLOUD
        INNER JOIN
            MES_PREVIO
        ON
            FECHA BETWEEN FECHA_INICIO AND FECHA_FIN
    ),

    ACTIVOS_ANIO_PREVIO AS (
        SELECT
            count(DISTINCT EMAIL) AS CLIENTES
        FROM
            TRANSACCIONES_OLO_Y_CLOUD
        INNER JOIN
            ANIO_PREVIO
        ON
            FECHA BETWEEN FECHA_INICIO AND FECHA_FIN
    )

    SELECT
        ACTIVOS_PERIODO_ACTUAL.CLIENTES AS SEMANA_ACTUAL,
        ACTIVOS_SEMANA_PREVIA.CLIENTES AS SEMANA_PREVIA,
        (SEMANA_ACTUAL - SEMANA_PREVIA) / SEMANA_PREVIA AS PERCENT_V_SEMANA_PREVIA,

        ACTIVOS_PERIODO_ACTUAL.CLIENTES AS MES_ACTUAL,
        ACTIVOS_MES_PREVIO.CLIENTES AS MES_PREVIO,
        (MES_ACTUAL - MES_PREVIO) / MES_PREVIO AS PERCENT_V_MES_PREVIO,

        ACTIVOS_PERIODO_ACTUAL.CLIENTES AS ANIO_ACTUAL,
        ACTIVOS_ANIO_PREVIO.CLIENTES AS ANIO_PREVIO,
        (ANIO_ACTUAL - ANIO_PREVIO) / ANIO_PREVIO AS PERCENT_V_ANIO_PREVIO
    FROM
        ACTIVOS_PERIODO_ACTUAL
    JOIN
        ACTIVOS_SEMANA_PREVIA
    JOIN  
        ACTIVOS_MES_PREVIO
    JOIN
        ACTIVOS_ANIO_PREVIO
    ;
""").fetch_pandas_all()

activos['DATO'] = 'ACTIVOS'

activos.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO,DATO
0,1985277,1965319,0.010155,1985277,1945357,0.020521,1985277,1913060,0.037749,ACTIVOS


## Recuperados Totales

In [8]:
recuperados_semanal = conn.cursor().execute(f"""
    WITH

    SEMANAS AS (
        SELECT
            ANIO_ALSEA,
            MES_ALSEA,
            SEM_ALSEA,
            to_date(max(FECHA)) AS FECHA_FIN_SEMANA_ACTUAL,
            FECHA_FIN_SEMANA_ACTUAL - 180 AS FECHA_INICIO_SEMANA_ACTUAL,
            FECHA_FIN_SEMANA_ACTUAL - 7 AS FECHA_FIN_SEMANA_PREVIA,
            FECHA_FIN_SEMANA_PREVIA - 180 AS FECHA_INICIO_SEMANA_PREVIA
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
        (
            ANIO_ALSEA = {ANIO_PREVIO}
        )
        OR
        (
            ANIO_ALSEA = {ANIO_ACTUAL}
            AND
            SEM_ALSEA <= {SEMANA_ACTUAL}
        )
        GROUP BY
            ANIO_ALSEA,
            MES_ALSEA,
            SEM_ALSEA
    ),

    TRANSACCIONES_OLO AS (
        SELECT DISTINCT
            lower(MXP.EMAIL) AS EMAIL,
            to_date(OLO.ORDER_DATE) AS FECHA
        FROM
        "SEGMENT_EVENTS"."DOMINOS_OLO"."MXPOWERSALESLOG" MXP
        INNER JOIN 
            "SEGMENT_EVENTS"."DOMINOS_OLO"."DPMSALES_FULL" OLO
        ON 
            OLO.ORDER_NUMBER = MXP.ORDERNUMBER AND OLO.LOCATION_CODE = MXP.STORENUMBER 
        AND 
            TO_CHAR(OLO.ORDER_DATE,'YYYY-MM-DD') = MXP.ORDERDATE
        WHERE 
            OLO.ORDER_STATUS_CODE = 4
        AND 
            OLO.LOCATION_CODE NOT IN ('13001' , '13006', '13021', '11000')
        AND 
            UPPER(OLO.SOURCE_CODE) IN ('ANDROID' , 'DESKTOP', 'IOS', 'MOBILE', 'WEB', 'ANDROID2', 'DESKTOP2', 'IOSAPP', 'MOBILE2', 'WHATSAPP')
    ),

    TRANSACCIONES_CLOUD AS (
        SELECT DISTINCT 
            lower(A.EMAIL) AS EMAIL,
            to_date(SUBSTRING(A.STOREORDERID,1,10)) AS FECHA
        FROM 
            "SEGMENT_EVENTS"."DOMINOS_GOLO"."VENTA_CLOUD" A
        WHERE 
            A.STOREID NOT LIKE '9%'
        AND 
            A.SOURCEORGANIZATIONURI IN ('order.dominos.com','resp-order.dominos.com','iphone.dominos.mx','android.dominos.mx') 
        AND 
            A.SOURCEORGANIZATIONURI IS NOT NULL
    ),

    TRANSACCIONES_TOTAL AS (
        SELECT * FROM TRANSACCIONES_CLOUD
        UNION ALL
        SELECT * FROM TRANSACCIONES_OLO
    ),

    PRIMERAS_COMPRAS AS (
        SELECT
            lower(EMAIL) AS EMAIL,
            min(FECHA) AS FECHA
        FROM
            TRANSACCIONES_TOTAL
        GROUP BY
            EMAIL
    ),

    PRIMERAS_COMPRAS_CON_SEMANA AS (
        SELECT
            ANIO_ALSEA,
            MES_ALSEA,
            SEM_ALSEA,
            EMAIL
        FROM
            PRIMERAS_COMPRAS
        INNER JOIN
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        USING(
            FECHA
        )
    ),

    ACTIVOS_EN_SEMANA_PREVIA AS (
        SELECT
            EMAIL,
            SEM_ALSEA,
            MES_ALSEA,
            ANIO_ALSEA
        FROM
            TRANSACCIONES_TOTAL
        INNER JOIN
            SEMANAS
        ON
            FECHA_INICIO_SEMANA_PREVIA < FECHA
        AND
            FECHA <= FECHA_FIN_SEMANA_PREVIA 
        GROUP BY
            EMAIL,
            SEM_ALSEA,
            MES_ALSEA,
            ANIO_ALSEA
    ),

    ACTIVOS_EN_SEMANA_ACTUAL AS (
        SELECT
            EMAIL,
            SEM_ALSEA,
            MES_ALSEA,
            ANIO_ALSEA
        FROM
            TRANSACCIONES_TOTAL
        INNER JOIN
            SEMANAS
        ON
            FECHA_INICIO_SEMANA_ACTUAL < FECHA
        AND
            FECHA <= FECHA_FIN_SEMANA_ACTUAL 
        GROUP BY
            EMAIL,
            SEM_ALSEA,
            MES_ALSEA,
            ANIO_ALSEA
    ),

    PRE_PIVOT AS (
        SELECT
            coalesce(ACTIVOS_EN_SEMANA_PREVIA.ANIO_ALSEA, ACTIVOS_EN_SEMANA_ACTUAL.ANIO_ALSEA) AS ANIO,
            coalesce(ACTIVOS_EN_SEMANA_PREVIA.MES_ALSEA, ACTIVOS_EN_SEMANA_ACTUAL.MES_ALSEA) AS MES,
            coalesce(ACTIVOS_EN_SEMANA_PREVIA.SEM_ALSEA, ACTIVOS_EN_SEMANA_ACTUAL.SEM_ALSEA) AS SEM,
            CASE
                WHEN ACTIVOS_EN_SEMANA_PREVIA.EMAIL IS null AND PRIMERAS_COMPRAS_CON_SEMANA.EMAIL IS null THEN 'RECUPERADOS'
                WHEN ACTIVOS_EN_SEMANA_PREVIA.EMAIL IS null THEN 'PRIMERAS_COMPRAS'
                WHEN ACTIVOS_EN_SEMANA_ACTUAL.EMAIL IS null THEN 'RECIEN_INACTIVADOS'
                ELSE 'ACTIVOS_EN_AMBOS_PERIODOS'
            END AS CATEGORIA,
            count(DISTINCT lower(coalesce(ACTIVOS_EN_SEMANA_PREVIA.EMAIL, ACTIVOS_EN_SEMANA_ACTUAL.EMAIL))) AS USUARIOS
        FROM
            ACTIVOS_EN_SEMANA_ACTUAL
        FULL OUTER JOIN
            ACTIVOS_EN_SEMANA_PREVIA
        USING(
            EMAIL,
            SEM_ALSEA,
            MES_ALSEA,
            ANIO_ALSEA
        )
        FULL OUTER JOIN
            PRIMERAS_COMPRAS_CON_SEMANA
        USING(
            EMAIL,
            SEM_ALSEA,
            MES_ALSEA,
            ANIO_ALSEA
        )
        GROUP BY
            SEM,
            MES,
            ANIO,
            CATEGORIA
    )

    SELECT
        *
    FROM
        PRE_PIVOT
    PIVOT(sum(USUARIOS) FOR CATEGORIA IN ('RECUPERADOS','PRIMERAS_COMPRAS','RECIEN_INACTIVADOS'))
    AS PIVOTED (ANIO_ALSEA, MES_ALSEA, SEM_ALSEA, RECUPERADOS, PRIMERAS_COMPRAS, RECIEN_INACTIVADOS)
    ORDER BY
        ANIO_ALSEA DESC,
        SEM_ALSEA DESC
    ;
""").fetch_pandas_all()

semana_actual = recuperados_semanal[(recuperados_semanal['SEM_ALSEA'] == SEMANA_ACTUAL) & (recuperados_semanal['ANIO_ALSEA'] == ANIO_ACTUAL       )]['RECUPERADOS'].iloc[0]
semana_previa = recuperados_semanal[(recuperados_semanal['SEM_ALSEA'] == SEMANA_PREVIA) & (recuperados_semanal['ANIO_ALSEA'] == ANIO_SEMANA_PREVIA)]['RECUPERADOS'].iloc[0]
percent_v_semana_previa = (semana_actual - semana_previa) / semana_previa

mes_actual = recuperados_semanal[(recuperados_semanal['MES_ALSEA'] == MES_ACTUAL) & (recuperados_semanal['SEM_ALSEA'] <= SEMANA_ACTUAL) & (recuperados_semanal['ANIO_ALSEA'] == ANIO_ACTUAL)].groupby(['ANIO_ALSEA', 'MES_ALSEA'])['RECUPERADOS'].sum().iloc[0]
mes_previo = recuperados_semanal[(recuperados_semanal['MES_ALSEA'] == MES_PREVIO) & (recuperados_semanal['ANIO_ALSEA'] == ANIO_MES_PREVIO)].groupby(['ANIO_ALSEA', 'MES_ALSEA'])['RECUPERADOS'].sum().iloc[0]
percent_v_mes_previo = (mes_actual - mes_previo) / mes_previo

anio_actual = recuperados_semanal[(recuperados_semanal['ANIO_ALSEA'] == ANIO_ACTUAL) & (recuperados_semanal['SEM_ALSEA'] <= SEMANA_ACTUAL)].groupby('ANIO_ALSEA')['RECUPERADOS'].sum().iloc[0]
anio_previo = recuperados_semanal[(recuperados_semanal['ANIO_ALSEA'] == ANIO_PREVIO) & (recuperados_semanal['SEM_ALSEA'] <= SEMANA_ACTUAL)].groupby('ANIO_ALSEA')['RECUPERADOS'].sum().iloc[0]
percent_v_anio_previo = (anio_actual - anio_previo) / anio_previo 

data = {
    'SEMANA_ACTUAL' : [semana_actual],
    'SEMANA_PREVIA' : [semana_previa],
    'PERCENT_V_SEMANA_PREVIA' : [percent_v_semana_previa],
    'MES_ACTUAL' : [mes_actual],
    'MES_PREVIO' : [mes_previo],
    'PERCENT_V_MES_PREVIO' : [percent_v_mes_previo],
    'ANIO_ACTUAL' : [anio_actual],
    'ANIO_PREVIO' : [anio_previo],
    'PERCENT_V_ANIO_PREVIO' : [percent_v_anio_previo],
    'DATO' : 'RECUPERADOS_TOTALES'
}

recuperados_totales = DataFrame(data)

recuperados_totales.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO,DATO
0,28096,25253,0.1125806834831505167702847186,53349,79377,-0.3279035488869571790317094372,942125,806162,0.1686546872712928666942872524,RECUPERADOS_TOTALES


### Existentes

In [9]:
semana_actual = activos['SEMANA_ACTUAL'].iloc[0] - recuperados_totales['SEMANA_ACTUAL'].iloc[0] - primeras_compras['SEMANA_ACTUAL'].iloc[0]
semana_previa = activos['SEMANA_PREVIA'].iloc[0] - recuperados_totales['SEMANA_PREVIA'].iloc[0] - primeras_compras['SEMANA_PREVIA'].iloc[0]
percent_v_semana_previa = (semana_actual - semana_previa) / semana_previa

mes_actual = activos['MES_ACTUAL'].iloc[0] - recuperados_totales['MES_ACTUAL'].iloc[0] - primeras_compras['MES_ACTUAL'].iloc[0]
mes_previo = activos['MES_PREVIO'].iloc[0] - recuperados_totales['MES_PREVIO'].iloc[0] - primeras_compras['MES_PREVIO'].iloc[0]
percent_v_mes_previo = (mes_actual - mes_previo) / mes_previo

anio_actual = activos['ANIO_ACTUAL'].iloc[0] - recuperados_totales['ANIO_ACTUAL'].iloc[0] - primeras_compras['ANIO_ACTUAL'].iloc[0]
anio_previo = activos['ANIO_PREVIO'].iloc[0] - recuperados_totales['ANIO_PREVIO'].iloc[0] - primeras_compras['ANIO_PREVIO'].iloc[0]
if anio_previo != 0:
    percent_v_anio_previo = (anio_actual - anio_previo) / anio_previo 
else:
    percent_v_anio_previo = 0

data = {
    'SEMANA_ACTUAL' : [semana_actual],
    'SEMANA_PREVIA' : [semana_previa],
    'PERCENT_V_SEMANA_PREVIA' : [percent_v_semana_previa],
    'MES_ACTUAL' : [mes_actual],
    'MES_PREVIO' : [mes_previo],
    'PERCENT_V_MES_PREVIO' : [percent_v_mes_previo],
    'ANIO_ACTUAL' : [anio_actual],
    'ANIO_PREVIO' : [anio_previo],
    'PERCENT_V_ANIO_PREVIO' : [percent_v_anio_previo],
    'DATO' : 'EXISTENTES'
}

existentes = DataFrame(data)

existentes.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO,DATO
0,1919757,1910166,0.005021029585910334494488960645,1864604,1765109,0.05636762375581338036347896929,-9964,-9919,0.004536747656013711059582619216,EXISTENTES


### Recien inactivados

In [10]:
semana_actual = recuperados_semanal[(recuperados_semanal['SEM_ALSEA'] == SEMANA_ACTUAL) & (recuperados_semanal['ANIO_ALSEA'] == ANIO_ACTUAL       )]['RECIEN_INACTIVADOS'].iloc[0]
semana_previa = recuperados_semanal[(recuperados_semanal['SEM_ALSEA'] == SEMANA_PREVIA) & (recuperados_semanal['ANIO_ALSEA'] == ANIO_SEMANA_PREVIA)]['RECIEN_INACTIVADOS'].iloc[0]
percent_v_semana_previa = (semana_actual - semana_previa) / semana_previa

mes_actual = recuperados_semanal[(recuperados_semanal['MES_ALSEA'] == MES_ACTUAL) & (recuperados_semanal['SEM_ALSEA'] <= SEMANA_ACTUAL) & (recuperados_semanal['ANIO_ALSEA'] == ANIO_ACTUAL)].groupby(['ANIO_ALSEA', 'MES_ALSEA'])['RECIEN_INACTIVADOS'].sum().iloc[0]
mes_previo = recuperados_semanal[(recuperados_semanal['MES_ALSEA'] == MES_PREVIO) & (recuperados_semanal['ANIO_ALSEA'] == ANIO_MES_PREVIO)].groupby(['ANIO_ALSEA', 'MES_ALSEA'])['RECIEN_INACTIVADOS'].sum().iloc[0]
percent_v_mes_previo = (mes_actual - mes_previo) / mes_previo

anio_actual = recuperados_semanal[(recuperados_semanal['ANIO_ALSEA'] == ANIO_ACTUAL) & (recuperados_semanal['SEM_ALSEA'] <= SEMANA_ACTUAL)].groupby('ANIO_ALSEA')['RECIEN_INACTIVADOS'].sum().iloc[0]
anio_previo = recuperados_semanal[(recuperados_semanal['ANIO_ALSEA'] == ANIO_PREVIO) & (recuperados_semanal['SEM_ALSEA'] <= SEMANA_ACTUAL)].groupby('ANIO_ALSEA')['RECIEN_INACTIVADOS'].sum().iloc[0]
percent_v_anio_previo = (anio_actual - anio_previo) / anio_previo 

data = {
    'SEMANA_ACTUAL' : [semana_actual],
    'SEMANA_PREVIA' : [semana_previa],
    'PERCENT_V_SEMANA_PREVIA' : [percent_v_semana_previa],
    'MES_ACTUAL' : [mes_actual],
    'MES_PREVIO' : [mes_previo],
    'PERCENT_V_MES_PREVIO' : [percent_v_mes_previo],
    'ANIO_ACTUAL' : [anio_actual],
    'ANIO_PREVIO' : [anio_previo],
    'PERCENT_V_ANIO_PREVIO' : [percent_v_anio_previo],
    'DATO' : 'RECIEN_INACTIVADOS'
}

recien_inactivados = DataFrame(data)

recien_inactivados.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO,DATO
0,45286,35945,0.2598692446793712616497426624,81231,163116,-0.5020047083057456043551828147,1882290,2051102,-0.08230307415233372109236888268,RECIEN_INACTIVADOS


### Inactivos

In [11]:
inactivos = conn.cursor().execute(f"""
    WITH

    PERIODO_ACTUAL AS (
        SELECT
            to_date(max(FECHA)) AS FECHA_FIN,
            FECHA_FIN - 180 AS FECHA_INICIO_ACTIVOS,
            FECHA_FIN - 365 AS FECHA_INICIO_INACTIVOS
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA = {SEMANA_ACTUAL}
    ),

    SEMANA_PREVIA AS (
        SELECT
            to_date(max(FECHA)) AS FECHA_FIN,
            FECHA_FIN - 180 AS FECHA_INICIO_ACTIVOS,
            FECHA_FIN - 365 AS FECHA_INICIO_INACTIVOS
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            SEM_ALSEA = {SEMANA_PREVIA}
    ),

    MES_PREVIO AS (
        SELECT
            to_date(max(FECHA)) AS FECHA_FIN,
            FECHA_FIN - 180 AS FECHA_INICIO_ACTIVOS,
            FECHA_FIN - 365 AS FECHA_INICIO_INACTIVOS
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
            ANIO_ALSEA = {ANIO_ACTUAL}
        AND
            MES_ALSEA = {MES_PREVIO}
    ),

    ANIO_PREVIO AS (
        SELECT
            to_date(max(FECHA)) AS FECHA_FIN,
            FECHA_FIN - 180 AS FECHA_INICIO_ACTIVOS,
            FECHA_FIN - 365 AS FECHA_INICIO_INACTIVOS
        FROM
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        WHERE
            ANIO_ALSEA = {ANIO_PREVIO}
        AND
            SEM_ALSEA = {SEMANA_ACTUAL}
    ),                                      

    TRANSACCIONES_OLO AS (
        SELECT DISTINCT
            lower(MXP.EMAIL) AS EMAIL,
            ORDER_DATE AS FECHA,
            TO_CHAR(ORDER_DATE,'YYYY-MM-DD') || '-' || LOCATION_CODE || '-' || OLO.ORDER_NUMBER AS ORDER_ID,
            OLO.ORDERFINALPRICE / 1.16 AS ORDER_AMOUNT,
            PHONENUMBER AS PHONE,
            FIRSTNAME
        FROM
        "SEGMENT_EVENTS"."DOMINOS_OLO"."MXPOWERSALESLOG" MXP
        INNER JOIN 
            "SEGMENT_EVENTS"."DOMINOS_OLO"."DPMSALES_FULL" OLO
        ON 
            OLO.ORDER_NUMBER = MXP.ORDERNUMBER AND OLO.LOCATION_CODE = MXP.STORENUMBER 
        AND 
            TO_CHAR(OLO.ORDER_DATE,'YYYY-MM-DD') = MXP.ORDERDATE
        WHERE 
            OLO.ORDER_STATUS_CODE = 4
        AND 
            OLO.LOCATION_CODE NOT IN ('13001' , '13006', '13021', '11000')
        AND 
            UPPER(OLO.SOURCE_CODE) IN (
                'ANDROID', 
                'DESKTOP', 
                'IOS', 
                'MOBILE', 
                'WEB', 
                'ANDROID2', 
                'DESKTOP2', 
                'IOSAPP', 
                'MOBILE2', 
                'WHATSAPP'
            )
    ),

    TRANSACCIONES_CLOUD AS (
        SELECT DISTINCT 
            lower(A.EMAIL) AS EMAIL,
            to_date(SUBSTRING(A.STOREORDERID,1,10)) AS FECHA,
            split_part(A.STOREORDERID, '#', 1) || '-' || A.STOREID || '-' || split_part(A.STOREORDERID, '#', 2) AS ORDER_ID,
            PAYMENTSAMOUNT / 1.16 AS ORDER_AMOUNT,
            PHONE,
            FIRSTNAME
        FROM 
            "SEGMENT_EVENTS"."DOMINOS_GOLO"."VENTA_CLOUD" A
        WHERE 
            A.STOREID NOT LIKE '9%'
        AND 
            A.SOURCEORGANIZATIONURI IN (
                'order.dominos.com', 
                'resp-order.dominos.com', 
                'iphone.dominos.mx', 
                'android.dominos.mx'
            )
    ),

    TRANSACCIONES_OLO_Y_CLOUD AS (
        SELECT * FROM TRANSACCIONES_OLO
        UNION ALL
        SELECT * FROM TRANSACCIONES_CLOUD
    ),

    BASE_INACTIVOS_PERIODO_ACTUAL AS (
        SELECT
            EMAIL
        FROM
            TRANSACCIONES_OLO_Y_CLOUD
        INNER JOIN
            PERIODO_ACTUAL
        ON
            FECHA < FECHA_FIN
        GROUP BY
            EMAIL,
            FECHA_INICIO_ACTIVOS,
            FECHA_INICIO_INACTIVOS
        HAVING
            max(FECHA) < FECHA_INICIO_ACTIVOS
        AND
            FECHA_INICIO_INACTIVOS <= max(FECHA)
    ),

    INACTIVOS_PERIODO_ACTUAL AS (
        SELECT
            count(*) AS CLIENTES
        FROM
            BASE_INACTIVOS_PERIODO_ACTUAL
    ),

    BASE_INACTIVOS_SEMANA_PREVIA AS (
        SELECT
            EMAIL
        FROM
            TRANSACCIONES_OLO_Y_CLOUD
        INNER JOIN
            SEMANA_PREVIA
        ON
            FECHA < FECHA_FIN
        GROUP BY
            EMAIL,
            FECHA_INICIO_ACTIVOS,
            FECHA_INICIO_INACTIVOS
        HAVING
            max(FECHA) < FECHA_INICIO_ACTIVOS
        AND
            FECHA_INICIO_INACTIVOS <= max(FECHA)
    ),

    INACTIVOS_SEMANA_PREVIA AS (
        SELECT
            count(*) AS CLIENTES
        FROM
            BASE_INACTIVOS_SEMANA_PREVIA
    ),

    BASE_INACTIVOS_MES_PREVIO AS (
        SELECT
            EMAIL
        FROM
            TRANSACCIONES_OLO_Y_CLOUD
        INNER JOIN
            MES_PREVIO
        ON
            FECHA < FECHA_FIN
        GROUP BY
            EMAIL,
            FECHA_INICIO_ACTIVOS,
            FECHA_INICIO_INACTIVOS
        HAVING
            max(FECHA) < FECHA_INICIO_ACTIVOS
        AND
            FECHA_INICIO_INACTIVOS <= max(FECHA)
    ),

    INACTIVOS_MES_PREVIO AS (
        SELECT
            count(*) AS CLIENTES
        FROM
            BASE_INACTIVOS_MES_PREVIO  
    ),

    BASE_INACTIVOS_ANIO_PREVIO AS (
        SELECT
            EMAIL
        FROM
            TRANSACCIONES_OLO_Y_CLOUD
        INNER JOIN
            ANIO_PREVIO
        ON
            FECHA < FECHA_FIN
        GROUP BY
            EMAIL,
            FECHA_INICIO_ACTIVOS,
            FECHA_INICIO_INACTIVOS
        HAVING
            max(FECHA) < FECHA_INICIO_ACTIVOS
        AND
            FECHA_INICIO_INACTIVOS <= max(FECHA)
    ),

    INACTIVOS_ANIO_PREVIO AS (
        SELECT
            count(*) AS CLIENTES
        FROM
            BASE_INACTIVOS_ANIO_PREVIO  
    )

    SELECT
        INACTIVOS_PERIODO_ACTUAL.CLIENTES AS SEMANA_ACTUAL,
        INACTIVOS_SEMANA_PREVIA.CLIENTES AS SEMANA_PREVIA,
        (SEMANA_ACTUAL - SEMANA_PREVIA) / SEMANA_PREVIA AS PERCENT_V_SEMANA_PREVIA,

        INACTIVOS_PERIODO_ACTUAL.CLIENTES AS MES_ACTUAL,
        INACTIVOS_MES_PREVIO.CLIENTES AS MES_PREVIO,
        (MES_ACTUAL - MES_PREVIO) / MES_PREVIO AS PERCENT_V_MES_PREVIO,

        INACTIVOS_PERIODO_ACTUAL.CLIENTES AS ANIO_ACTUAL,
        INACTIVOS_ANIO_PREVIO.CLIENTES AS ANIO_PREVIO,
        (ANIO_ACTUAL - ANIO_PREVIO) / ANIO_PREVIO AS PERCENT_V_ANIO_PREVIO
    FROM
        INACTIVOS_PERIODO_ACTUAL
    JOIN
        INACTIVOS_SEMANA_PREVIA
    JOIN  
        INACTIVOS_MES_PREVIO
    JOIN
        INACTIVOS_ANIO_PREVIO
        ;
""").fetch_pandas_all()

inactivos['DATO'] = 'INACTIVOS'

inactivos.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO,DATO
0,931287,923235,0.008722,931287,931766,-0.000514,931287,1094249,-0.148926,INACTIVOS


### Clientes totales

In [12]:
semana_actual = activos['SEMANA_ACTUAL'].iloc[0] + inactivos['SEMANA_ACTUAL'].iloc[0]
semana_previa = activos['SEMANA_PREVIA'].iloc[0] + inactivos['SEMANA_PREVIA'].iloc[0]
percent_v_semana_previa = (semana_actual - semana_previa) / semana_previa

mes_actual = activos['MES_ACTUAL'].iloc[0] + inactivos['MES_ACTUAL'].iloc[0]
mes_previo = activos['MES_PREVIO'].iloc[0] + inactivos['MES_PREVIO'].iloc[0]
percent_v_mes_previo = (mes_actual - mes_previo) / mes_previo

anio_actual = activos['ANIO_ACTUAL'].iloc[0] + inactivos['ANIO_ACTUAL'].iloc[0]
anio_previo = activos['ANIO_PREVIO'].iloc[0] + inactivos['ANIO_PREVIO'].iloc[0]
if anio_previo != 0:
    percent_v_anio_previo = (anio_actual - anio_previo) / anio_previo 
else:
    percent_v_anio_previo = 0

data = {
    'SEMANA_ACTUAL' : [semana_actual],
    'SEMANA_PREVIA' : [semana_previa],
    'PERCENT_V_SEMANA_PREVIA' : [percent_v_semana_previa],
    'MES_ACTUAL' : [mes_actual],
    'MES_PREVIO' : [mes_previo],
    'PERCENT_V_MES_PREVIO' : [percent_v_mes_previo],
    'ANIO_ACTUAL' : [anio_actual],
    'ANIO_PREVIO' : [anio_previo],
    'PERCENT_V_ANIO_PREVIO' : [percent_v_anio_previo],
    'DATO' : 'CLIENTES_TOTALES'
}

clientes_totales = DataFrame(data)

clientes_totales.head()

,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO,DATO
0,2916564,2888554,0.009697,2916564,2877123,0.013708,2916564,3007309,-0.030175,CLIENTES_TOTALES


In [13]:
from datetime import date

recuperados_campania = DataFrame({
    'DATO': ['RECUPERADOS_CAMPANIA'], 
    'SEMANA_ACTUAL': [''], 'SEMANA_PREVIA': [''], 'PERCENT_V_SEMANA_PREVIA': [''], 'TASAS_SEMANALES': [''],
    'MES_ACTUAL': [''], 'MES_PREVIO': [''], 'PERCENT_V_MES_PREVIO': [''], 'TASAS_MENSUALES': [''],
    'ANIO_ACTUAL': [''], 'ANIO_PREVIO': [''], 'PERCENT_V_ANIO_PREVIO': [''], 'TASAS_ANUALES': ['']
})

recuperados_organicos = DataFrame({
    'DATO': ['RECUPERADOS_ORGANICOS'], 
    'SEMANA_ACTUAL': [''], 'SEMANA_PREVIA': [''], 'PERCENT_V_SEMANA_PREVIA': [''], 'TASAS_SEMANALES': [''],
    'MES_ACTUAL': [''], 'MES_PREVIO': [''], 'PERCENT_V_MES_PREVIO': [''], 'TASAS_MENSUALES': [''],
    'ANIO_ACTUAL': [''], 'ANIO_PREVIO': [''], 'PERCENT_V_ANIO_PREVIO': [''], 'TASAS_ANUALES': ['']
})

datos_totales = concat([instalaciones, primeras_compras, registros_totales, registros_con_compra, registros_sin_compra, activos, existentes, recuperados_totales, recuperados_campania, recuperados_organicos, inactivos, recien_inactivados, clientes_totales])

dato_activos = activos.get('SEMANA_ACTUAL')[0]

tasa_adquisicion_semanal = primeras_compras.get('SEMANA_ACTUAL')[0] / dato_activos
tasa_recuperada_semanal = recuperados_totales.get('SEMANA_ACTUAL')[0] / dato_activos
tasa_perdida_semanal = recien_inactivados.get('SEMANA_ACTUAL')[0] / dato_activos
tasa_total_semanal = tasa_adquisicion_semanal + float(tasa_recuperada_semanal) - float(tasa_perdida_semanal)
datos_totales['TASAS_SEMANALES'] = ['Tasa Adquisicion:', tasa_adquisicion_semanal, '', '', '', '', 'Tasa recuperada:', tasa_recuperada_semanal, '', '', 'Tasa perdida:', tasa_perdida_semanal, tasa_total_semanal]

sem_inicial_mes = conn.cursor().execute(f"""
    SELECT
        min(SEM_ALSEA) AS SEMANA
    FROM
        WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
    WHERE
        ANIO_ALSEA = {ANIO_ACTUAL}
    AND
        MES_ALSEA = {MES_ACTUAL}
    GROUP BY
        ANIO_ALSEA,
        MES_ALSEA
""").fetch_pandas_all()

semanas_en_mes = SEMANA_ACTUAL - sem_inicial_mes.get('SEMANA')[0] + 1

tasa_adquisicion_mensual = primeras_compras.get('MES_ACTUAL')[0] / (dato_activos * semanas_en_mes)
tasa_recuperada_mensual = recuperados_totales.get('MES_ACTUAL')[0] / (dato_activos * semanas_en_mes)
tasa_perdida_mensual = recien_inactivados.get('MES_ACTUAL')[0] / (dato_activos * semanas_en_mes)
tasa_total_mensual = tasa_adquisicion_mensual + float(tasa_recuperada_mensual) - float(tasa_perdida_mensual)
datos_totales['TASAS_MENSUALES'] = ['Tasa Adquisicion:', tasa_adquisicion_mensual, '', '', '', '', 'Tasa recuperada:', tasa_recuperada_mensual, '', '', 'Tasa perdida:', tasa_perdida_mensual, tasa_total_mensual]

tasa_adquisicion_anual = primeras_compras.get('ANIO_ACTUAL')[0] / (dato_activos * SEMANA_ACTUAL)
tasa_recuperada_anual = recuperados_totales.get('ANIO_ACTUAL')[0] / (dato_activos * SEMANA_ACTUAL)
tasa_perdida_anual = recien_inactivados.get('ANIO_ACTUAL')[0] / (dato_activos * SEMANA_ACTUAL)
tasa_total_anual = tasa_adquisicion_anual + float(tasa_recuperada_anual) - float(tasa_perdida_anual)
datos_totales['TASAS_ANUALES'] = ['Tasa Adquisicion:', tasa_adquisicion_anual, '', '', '', '', 'Tasa recuperada:', tasa_recuperada_anual, '', '', 'Tasa perdida:', tasa_perdida_anual, tasa_total_anual]

datos_totales['META'] = ''

datos_totales['TASAS_META'] = ''

datos_totales = datos_totales[[
    'DATO', 
    'SEMANA_ACTUAL', 'SEMANA_PREVIA', 'PERCENT_V_SEMANA_PREVIA', 'TASAS_SEMANALES',
    'MES_ACTUAL', 'MES_PREVIO', 'PERCENT_V_MES_PREVIO', 'TASAS_MENSUALES',
    'ANIO_ACTUAL', 'ANIO_PREVIO', 'PERCENT_V_ANIO_PREVIO', 'TASAS_ANUALES',
    'META', 'TASAS_META'
]]

datos_totales.to_csv(f'entregas_semanales_dp/{date.today()}_foto_datos_{ANIO_ACTUAL}W{SEMANA_ACTUAL}.csv', index = False)

datos_totales

,DATO,SEMANA_ACTUAL,SEMANA_PREVIA,PERCENT_V_SEMANA_PREVIA,TASAS_SEMANALES,MES_ACTUAL,MES_PREVIO,PERCENT_V_MES_PREVIO,TASAS_MENSUALES,ANIO_ACTUAL,ANIO_PREVIO,PERCENT_V_ANIO_PREVIO,TASAS_ANUALES,META,TASAS_META
0,INSTALACIONES,136646,127620,0.070726,Tasa Adquisicion:,264266,300085,-0.119363,Tasa Adquisicion:,1976277,3115509,-0.365665,Tasa Adquisicion:,,
0,PRIMERAS_COMPRAS,37424,29900,0.251639,0.018851,67324,100871,-0.332573,0.016956,1053116,1116817,-0.057038,0.011788,,
0,REGISTROS_TOTALES,17167,15111,0.136060,,32269,72881,-0.557237,,340723,0,0.000000,,,
0,REGISTROS_CON_COMPRA,10708,8669,0.235206,,19695,40023,-0.507908,,191939,0,0.000000,,,
0,REGISTROS_SIN_COMPRA,6459,6442,0.002639,,12574,32858,-0.617323,,148784,0,0,,,
0,ACTIVOS,1985277,1965319,0.010155,,1985277,1945357,0.020521,,1985277,1913060,0.037749,,,
0,EXISTENTES,1919757,1910166,0.005021029585910334494488960645,Tasa recuperada:,1864604,1765109,0.05636762375581338036347896929,Tasa recuperada:,-9964,-9919,0.004536747656013711059582619216,Tasa recuperada:,,
0,RECUPERADOS_TOTALES,28096,25253,0.1125806834831505167702847186,0.01415218128251120624477088084,53349,79377,-0.3279035488869571790317094372,0.01343616029400431274829658531,942125,806162,0.1686546872712928666942872524,0.01054568763508120585243827995,,
0,RECUPERADOS_CAMPANIA,,,,,,,,,,,,,,
0,RECUPERADOS_ORGANICOS,,,,,,,,,,,,,,


## Cupones generados

In [14]:
from DP_cupones_generados_semanales import get_cupones_generados_semanales
from snowflake.snowpark.functions import col, count_distinct

cupones_generados_semanales = (
    get_cupones_generados_semanales(session)
    .order_by(col('SEM_ALSEA'))
    .filter((col('ANIO_ALSEA') < ANIO_ACTUAL) | ((col('SEM_ALSEA') <= SEMANA_ACTUAL) & (col('ANIO_ALSEA') == ANIO_ACTUAL)))
)

(
    cupones_generados_semanales
    .group_by(['COUPON'])
    .agg(count_distinct(col('CUPON')).alias('ENVIOS'))
    .to_pandas()
    .to_csv(f'./entregas_semanales_dp/{date.today()}_cupones_generados_total_{ANIO_ACTUAL}W{SEMANA_ACTUAL}.csv', index = False)
)

(
    cupones_generados_semanales
    .group_by(['ANIO_ALSEA', 'SEM_ALSEA', 'COUPON'])
    .agg(count_distinct(col('CUPON')).alias('ENVIOS'))
    .to_pandas()
    .pivot(index = ['ANIO_ALSEA', 'SEM_ALSEA'], columns = 'COUPON', values = 'ENVIOS')
    .to_csv(f'./entregas_semanales_dp/{date.today()}cupones_generados_semanales_{ANIO_ACTUAL}W{SEMANA_ACTUAL}.csv', index = False)
)

## Primeras compras con cupon GO199

In [15]:
redenciones = conn.cursor().execute("""
    WITH

    TRANSACCIONES_OLO AS (
        SELECT DISTINCT
            LOCATION_CODE || CUSTOMER_CODE AS ID_CLIENTE,
            to_date(MXP.ORDERDATE) AS FECHA,
            lower(EMAIL) AS EMAIL
        FROM
            "SEGMENT_EVENTS"."DOMINOS_OLO"."DPMSALES_FULL" OLO
        INNER JOIN
            "SEGMENT_EVENTS"."DOMINOS_OLO"."MXPOWERSALESLOG" MXP 
        ON 
            OLO.ORDER_NUMBER = MXP.ORDERNUMBER 
        AND 
            OLO.LOCATION_CODE = MXP.STORENUMBER 
        AND 
            TO_CHAR(OLO.ORDER_DATE,'YYYY-MM-DD') = MXP.ORDERDATE
        WHERE
            ORDER_STATUS_CODE = 4
        AND 
            SOURCE_CODE  IN ('ANDROID2','DESKTOP2','IOSAPP','DESKTOP','MOBILE2','ANDROID')
        AND 
            OLO.SOURCE_CODE IS NOT NULL
        AND 
            UPPER(COMMENTS) NOT LIKE '%DIDI%'
        AND 
            UPPER(COMMENTS) NOT LIKE '%UBER%'
        AND 
            UPPER(COMMENTS) NOT LIKE '%RAPPI%'
    ),

    FUNNEL AS (
        SELECT DISTINCT
            ID_CLIENTE,
            FDALTABD,
            FCCANAL,
            FECHA
        FROM
            SEGMENT_EVENTS.ALMODCOMUN.TACCLIENTES 
        INNER JOIN
        (
            SELECT DISTINCT
                concat(FCIDTIENDA, FIIDCLIENTE) AS ID_CLIENTE,
                to_date(FDIDDIA) AS FECHA
            FROM
                SEGMENT_EVENTS.ALOPERACIONES.TAFORDENES 
            WHERE
                concat(FCIDTIENDA, FITICKET) IN (
                    SELECT DISTINCT
                        concat(FCIDTIENDA, FITICKET) 
                    FROM
                        SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3 
                    WHERE
                        FCIDDESCUENTO IN (4647, 4648)
                )
            AND
                FIIDCLIENTE IS NOT null
            AND
                FIIDCLIENTE <> 0
        ) AS SUBQ
        ON
            SUBQ.ID_CLIENTE = concat (FIIDTIENDA, FIIDCLIENTE)
    ),

    EMAILS_OLO AS (
        SELECT DISTINCT
            EMAIL,
            FECHA
        FROM
            FUNNEL
        INNER JOIN
            TRANSACCIONES_OLO
        USING(
            ID_CLIENTE,
            FECHA
        )
    ),

    EMAILS_CLOUD AS (
        SELECT DISTINCT
            lower(EMAIL) AS EMAIL,
            FECHA
        FROM 
            "SEGMENT_EVENTS"."DOMINOS_GOLO"."VENTA_CLOUD" A
        INNER JOIN  
            WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
        ON
            to_date(SUBSTRING(A.STOREORDERID,1,10)) = FECHA
        WHERE
            A.STOREID NOT LIKE '9%'
        AND 
            A.SOURCEORGANIZATIONURI IN ('order.dominos.com','resp-order.dominos.com','iphone.dominos.mx','android.dominos.mx') 
        AND 
            A.SOURCEORGANIZATIONURI IS NOT NULL
        AND 
        (
            COUPONSCODE LIKE '%GO199%'
        )
    ),



    TRANSACCIONES_OLO_2 AS (
        SELECT DISTINCT
            lower(MXP.EMAIL) AS EMAIL,
            ORDER_DATE AS FECHA,
            TO_CHAR(ORDER_DATE,'YYYY-MM-DD') || '-' || LOCATION_CODE || '-' || OLO.ORDER_NUMBER AS ORDER_ID,
            OLO.ORDERFINALPRICE / 1.16 AS ORDER_AMOUNT,
            PHONENUMBER AS PHONE,
            FIRSTNAME
        FROM
        "SEGMENT_EVENTS"."DOMINOS_OLO"."MXPOWERSALESLOG" MXP
        INNER JOIN 
            "SEGMENT_EVENTS"."DOMINOS_OLO"."DPMSALES_FULL" OLO
        ON 
            OLO.ORDER_NUMBER = MXP.ORDERNUMBER AND OLO.LOCATION_CODE = MXP.STORENUMBER 
        AND 
            TO_CHAR(OLO.ORDER_DATE,'YYYY-MM-DD') = MXP.ORDERDATE
        WHERE 
            OLO.ORDER_STATUS_CODE = 4
        AND 
            OLO.LOCATION_CODE NOT IN ('13001' , '13006', '13021', '11000')
        AND 
            UPPER(OLO.SOURCE_CODE) IN (
                'ANDROID', 
                'DESKTOP', 
                'IOS', 
                'MOBILE', 
                'WEB', 
                'ANDROID2', 
                'DESKTOP2', 
                'IOSAPP', 
                'MOBILE2', 
                'WHATSAPP'
            )
    ),

    TRANSACCIONES_CLOUD AS (
        SELECT DISTINCT 
            lower(A.EMAIL) AS EMAIL,
            to_date(SUBSTRING(A.STOREORDERID,1,10)) AS FECHA,
            split_part(A.STOREORDERID, '#', 1) || '-' || A.STOREID || '-' || split_part(A.STOREORDERID, '#', 2) AS ORDER_ID,
            PAYMENTSAMOUNT / 1.16 AS ORDER_AMOUNT,
            PHONE,
            FIRSTNAME
        FROM 
            "SEGMENT_EVENTS"."DOMINOS_GOLO"."VENTA_CLOUD" A
        WHERE 
            A.STOREID NOT LIKE '9%'
        AND 
            A.SOURCEORGANIZATIONURI IN (
                'order.dominos.com', 
                'resp-order.dominos.com', 
                'iphone.dominos.mx', 
                'android.dominos.mx'
            )
    ),

    PRIMERAS_COMPRAS AS (
        SELECT
            EMAIL,
            min(FECHA) AS FECHA
        FROM
        (
            SELECT * FROM TRANSACCIONES_OLO_2
            UNION ALL
            SELECT * FROM TRANSACCIONES_CLOUD
        )
        GROUP BY
            EMAIL
    ),

    USUARIOS_CON_CUPON AS (
        SELECT
            *
        FROM
        (
            SELECT * FROM EMAILS_OLO
            UNION ALL
            SELECT * FROM EMAILS_CLOUD
        )
        INNER JOIN
            PRIMERAS_COMPRAS
        USING (
            EMAIL,
            FECHA
        )
    )


    SELECT
        ANIO_ALSEA,
        SEM_ALSEA,
        count(DISTINCT EMAIL)
    FROM
        USUARIOS_CON_CUPON
    INNER JOIN
        WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
    USING(
        FECHA
    )
    GROUP BY
        ANIO_ALSEA,
        SEM_ALSEA
    ;
""").fetch_pandas_all() 

redenciones = redenciones[redenciones['SEM_ALSEA'] <= SEMANA_ACTUAL]
redenciones.to_csv(f'entregas_semanales_dp/{date.today()}_primeras_compras_con_cupon_go199_{ANIO_ACTUAL}W{SEMANA_ACTUAL}.csv')

## Datos primeras compras adquisicion

In [16]:
from DP_transacciones_total import get_transacciones_olo, get_transacciones_cloud, get_transacciones_dp_total
import snowflake.snowpark.functions as fn

transacciones_dp_total = get_transacciones_dp_total(session)

descuentos_40_OFF = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3')
    .filter(fn.col('FCIDDESCUENTO').isin([4493, 4494]))
    .select(['FCIDTIENDA', 'FITICKET'])
)

ordenes_40_OFF = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFORDENES')
    .filter(fn.col('FIIDCLIENTE').is_not_null())
    .filter(fn.col('FIIDCLIENTE') != 0)
    .with_column('ID_CLIENTE', fn.concat(fn.col('FCIDTIENDA'), fn.col('FIIDCLIENTE')))
    .with_column('FECHA', fn.to_date('FDIDDIA'))
    .join(descuentos_40_OFF, on = ['FCIDTIENDA', 'FITICKET'], how = 'INNER')
)

clientes_40_OFF = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACCLIENTES')
    .with_column('ID_CLIENTE', fn.concat(fn.col('FIIDTIENDA'), fn.col('FIIDCLIENTE')))
    .join(ordenes_40_OFF, on = 'ID_CLIENTE')
)

transacciones_olo = get_transacciones_olo(session)

redenciones_olo_40_OFF = (
    transacciones_olo
    .join(clientes_40_OFF, on = ['ID_CLIENTE', 'FECHA'])
    .select(['EMAIL', 'FECHA'])
)

redenciones_cloud_40_OFF = (
    get_transacciones_cloud(session)
    .filter(fn.col('COUPONSCODE').like('%HELLO%'))
    .select(['EMAIL', 'FECHA'])
)

redenciones_totales_40_OFF = redenciones_cloud_40_OFF.union_all(redenciones_olo_40_OFF)

redenciones_semana_40_OFF = (
    redenciones_totales_40_OFF
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == ANIO_ACTUAL)
    .filter(fn.col('SEM_ALSEA') == SEMANA_ACTUAL)
    .with_column_renamed('FECHA', 'FECHA_ADQUISICION')
    .select('EMAIL', 'FECHA_ADQUISICION')
)

datos_recompras_semana_40_OFF = (
    redenciones_semana_40_OFF
    .join(transacciones_dp_total, on = 'EMAIL')
    .filter(fn.col('FECHA') >= fn.col('FECHA_ADQUISICION'))
    .group_by('EMAIL')
    .agg(
        fn.sum('VENTA').alias('VENTA'),
        fn.count_distinct('ORDER_ID').alias('TRANSACCIONES')
    )
    .order_by('TRANSACCIONES', ascending = False)
    .with_column('COHORT',
        fn.when(fn.col('TRANSACCIONES') > 3, 4)
        .otherwise(fn.col('TRANSACCIONES'))
    )
    .group_by('COHORT')
    .agg(
        fn.count_distinct('EMAIL').alias('USUARIOS'),
        fn.sum('VENTA').alias('VENTA'),
        fn.sum('TRANSACCIONES').alias('TRANSACCIONES')
    )
    .with_column('TICKET', fn.col('VENTA') / fn.col('TRANSACCIONES'))
    .with_column('FRECUENCIA', fn.col('TRANSACCIONES') / fn.col('USUARIOS'))
)

datos_recompras_semana_40_OFF_pd = datos_recompras_semana_40_OFF.to_pandas()
datos_recompras_semana_40_OFF_pd.to_csv(f'./entregas_semanales_dp/{date.today()}datos_recompras_40_OFF_{ANIO_ACTUAL}W{SEMANA_ACTUAL}.csv')
datos_recompras_semana_40_OFF_pd

,COHORT,USUARIOS,VENTA,TRANSACCIONES,TICKET,FRECUENCIA
0,1,1108,268417.758621,1108,242.254295,1.000000
1,4,3,3550.517241,14,253.608374,4.666667
2,2,54,23125.689655,108,214.126756,2.000000
3,3,2,1902.241379,6,317.040230,3.000000


## Datos primeras compras GO199

In [17]:
from DP_transacciones_total import get_transacciones_olo, get_transacciones_cloud, get_transacciones_dp_total
from general_get_tiempo import get_tiempo
from general_primeras_compras import get_primeras_compras

transacciones_dp_total = get_transacciones_dp_total(session)

tiempo = get_tiempo(session)

primeras_compras_semana_actual = (
    get_primeras_compras(transacciones_dp_total)
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == ANIO_ACTUAL)
    .filter(fn.col('SEM_ALSEA') == SEMANA_ACTUAL)
    .with_column_renamed('FECHA', 'FECHA_ADQUISICION')
)

descuentos_GO199 = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3')
    .filter(fn.col('FCIDDESCUENTO').isin([4647, 4648]))
    .select(['FCIDTIENDA', 'FITICKET'])
)

ordenes_GO199 = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFORDENES')
    .filter(fn.col('FIIDCLIENTE').is_not_null())
    .filter(fn.col('FIIDCLIENTE') != 0)
    .with_column('ID_CLIENTE', fn.concat(fn.col('FCIDTIENDA'), fn.col('FIIDCLIENTE')))
    .with_column('FECHA', fn.to_date('FDIDDIA'))
    .join(descuentos_GO199, on = ['FCIDTIENDA', 'FITICKET'], how = 'INNER')
)

clientes_GO199 = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACCLIENTES')
    .with_column('ID_CLIENTE', fn.concat(fn.col('FIIDTIENDA'), fn.col('FIIDCLIENTE')))
    .join(ordenes_GO199, on = 'ID_CLIENTE')
)

transacciones_olo = get_transacciones_olo(session)

redenciones_olo_GO199 = (
    transacciones_olo
    .join(clientes_GO199, on = ['ID_CLIENTE', 'FECHA'])
    .select(['EMAIL', 'FECHA'])
)

redenciones_cloud_GO199 = (
    get_transacciones_cloud(session)
    .filter(fn.col('COUPONSCODE').like(r'%GO199%'))
    .select(['EMAIL', 'FECHA'])
)

redenciones_totales_GO199 = redenciones_cloud_GO199.union_all(redenciones_olo_GO199)

redenciones_semana_GO199 = (
    redenciones_totales_GO199
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == ANIO_ACTUAL)
    .filter(fn.col('SEM_ALSEA') == SEMANA_ACTUAL)
    .with_column_renamed('FECHA', 'FECHA_ADQUISICION')
    .select('EMAIL', 'FECHA_ADQUISICION')
)

adquisiciones_semana_GO199 = (
    redenciones_semana_GO199
    .join(primeras_compras_semana_actual, on = ['FECHA_ADQUISICION', 'EMAIL'])
)

datos_recompras_semana_GO199 = (
    adquisiciones_semana_GO199
    .join(transacciones_dp_total, on = 'EMAIL')
    .filter(fn.col('FECHA') >= fn.col('FECHA_ADQUISICION'))
    .group_by('EMAIL')
    .agg(
        fn.sum('VENTA').alias('VENTA'),
        fn.count_distinct('ORDER_ID').alias('TRANSACCIONES')
    )
    .with_column('COHORT',
        fn.when(fn.col('TRANSACCIONES') > 3, 4)
        .otherwise(fn.col('TRANSACCIONES'))
    )
    .group_by('COHORT')
    .agg(
        fn.count_distinct('EMAIL').alias('USUARIOS'),
        fn.sum('VENTA').alias('VENTA'),
        fn.sum('TRANSACCIONES').alias('TRANSACCIONES')
    )
    .with_column('TICKET', fn.col('VENTA') / fn.col('TRANSACCIONES'))
    .with_column('FRECUENCIA', fn.col('TRANSACCIONES') / fn.col('USUARIOS'))
)

datos_recompras_semana_GO199_pd = datos_recompras_semana_GO199.to_pandas()
datos_recompras_semana_GO199_pd.to_csv(f'./entregas_semanales_dp/{date.today()}datos_recompras_GO199_{ANIO_ACTUAL}W{SEMANA_ACTUAL}.csv')
datos_recompras_semana_GO199_pd

,COHORT,USUARIOS,VENTA,TRANSACCIONES,TICKET,FRECUENCIA
0,1,2203,491198.103448,2203,222.967818,1.000000
1,4,3,6002.586207,20,300.129310,6.666667
2,2,165,95810.172414,330,290.333856,2.000000
3,3,18,16372.413793,54,303.192848,3.000000


## Datos primeras compras general

In [18]:
from DP_transacciones_total import get_transacciones_dp_total
from general_primeras_compras import get_primeras_compras
from general_get_tiempo import get_tiempo

tiempo = get_tiempo(session)

transacciones_dp_total = get_transacciones_dp_total(session)

primeras_compras = get_primeras_compras(transacciones_dp_total)

primeras_compras_semana = (
    primeras_compras
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == ANIO_ACTUAL)
    .filter(fn.col('SEM_ALSEA') == SEMANA_ACTUAL)
    .with_column_renamed('FECHA', 'FECHA_ADQUISICION')
    .select('EMAIL', 'FECHA_ADQUISICION')
)

datos_recompras_semana = (
    primeras_compras_semana
    .join(transacciones_dp_total, on = 'EMAIL')
    .filter(fn.col('FECHA') >= fn.col('FECHA_ADQUISICION'))
    .group_by('EMAIL')
    .agg(
        fn.sum('VENTA').alias('VENTA'),
        fn.count_distinct('ORDER_ID').alias('TRANSACCIONES')
    )
    .order_by('TRANSACCIONES', ascending = False)
    .with_column('COHORT',
        fn.when(fn.col('TRANSACCIONES') > 3, 4)
        .otherwise(fn.col('TRANSACCIONES'))
    )
    .group_by('COHORT')
    .agg(
        fn.count_distinct('EMAIL').alias('USUARIOS'),
        fn.sum('VENTA').alias('VENTA'),
        fn.sum('TRANSACCIONES').alias('TRANSACCIONES')
    )
    .with_column('TICKET', fn.col('VENTA') / fn.col('TRANSACCIONES'))
    .with_column('FRECUENCIA', fn.col('TRANSACCIONES') / fn.col('USUARIOS'))
)

datos_recompras_semana_pd = datos_recompras_semana.to_pandas()
datos_recompras_semana_pd.to_csv(f'./entregas_semanales_dp/{date.today()}datos_recompras_general_{ANIO_ACTUAL}W{SEMANA_ACTUAL}.csv')
datos_recompras_semana_pd

,COHORT,USUARIOS,VENTA,TRANSACCIONES,TICKET,FRECUENCIA
0,1,35238,8.414278e+06,35238,238.784206,1.000000
1,4,28,4.321638e+04,138,313.162169,4.928571
2,3,172,1.324931e+05,516,256.769664,3.000000
3,2,1986,9.446388e+05,3972,237.824480,2.000000


### Buscar cupones (en caso de que se necesite un cupon nuevo)

In [3]:
session.table('SEGMENT_EVENTS.ALMODCOMUN.TACDESCUENTOS').filter(fn.col('FIIDDESCUENTOSRC').like(r'%DS199O%')).show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FIIDMARCA"  |"FIIDDESCUENTO"  |"FIIDTIPODESCUENTO"  |"FIIDDESCUENTOSRC"  |"FCDESCUENTODESC"               |"FDALTABD"           |"FDULTACT"           |"FCUSER"       |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|193          |4623             |178                  |DS199O              |DS199O DominoÂ´s Mania 1-9 Ing  |2023-09-01 04:49:56  |2023-09-01 04:49:56  |Administrator  |
|193          |4625             |178                  |DS199O              |PIZZA 1-9 ING                   |2023-09-01 04:49:56  |2023-09-01 04:49:56  |Administrator  |
|4            |4626             |151                  |DS199O              |PIZZA 1-9 ING                   |2023-09-01 04:49:56  |2023-09-01 04:49:56

## Datos primeras compras cupon DS199O

In [19]:
from DP_transacciones_total import get_transacciones_olo, get_transacciones_cloud, get_transacciones_dp_total
from general_get_tiempo import get_tiempo
from general_primeras_compras import get_primeras_compras

transacciones_dp_total = get_transacciones_dp_total(session)

tiempo = get_tiempo(session)

primeras_compras_semana_actual = (
    get_primeras_compras(transacciones_dp_total)
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == ANIO_ACTUAL)
    .filter(fn.col('SEM_ALSEA') == SEMANA_ACTUAL)
    .with_column_renamed('FECHA', 'FECHA_ADQUISICION')
)

descuentos_DS199O = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3')
    .filter(fn.col('FCIDDESCUENTO').isin([4625, 4623, 4626]))
    .select(['FCIDTIENDA', 'FITICKET'])
)

ordenes_DS199O = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFORDENES')
    .filter(fn.col('FIIDCLIENTE').is_not_null())
    .filter(fn.col('FIIDCLIENTE') != 0)
    .with_column('ID_CLIENTE', fn.concat(fn.col('FCIDTIENDA'), fn.col('FIIDCLIENTE')))
    .with_column('FECHA', fn.to_date('FDIDDIA'))
    .join(descuentos_GO199, on = ['FCIDTIENDA', 'FITICKET'], how = 'INNER')
)

clientes_DS199O = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACCLIENTES')
    .with_column('ID_CLIENTE', fn.concat(fn.col('FIIDTIENDA'), fn.col('FIIDCLIENTE')))
    .join(ordenes_GO199, on = 'ID_CLIENTE')
)

transacciones_olo = get_transacciones_olo(session)

redenciones_olo_DS199O = (
    transacciones_olo
    .join(clientes_DS199O, on = ['ID_CLIENTE', 'FECHA'])
    .select(['EMAIL', 'FECHA'])
)

redenciones_cloud_DS199O = (
    get_transacciones_cloud(session)
    .filter(fn.col('COUPONSCODE').like(r'%DS199O%'))
    .select(['EMAIL', 'FECHA'])
)

redenciones_totales_DS199O = redenciones_cloud_DS199O.union_all(redenciones_olo_DS199O)

redenciones_semana_DS199O = (
    redenciones_totales_DS199O
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == ANIO_ACTUAL)
    .filter(fn.col('SEM_ALSEA') == SEMANA_ACTUAL)
    .with_column_renamed('FECHA', 'FECHA_ADQUISICION')
    .select('EMAIL', 'FECHA_ADQUISICION')
)

adquisiciones_semana_DS199O = (
    redenciones_semana_DS199O
    .join(primeras_compras_semana_actual, on = ['FECHA_ADQUISICION', 'EMAIL'])
)

datos_recompras_semana_DS199O = (
    adquisiciones_semana_DS199O
    .join(transacciones_dp_total, on = 'EMAIL')
    .filter(fn.col('FECHA') >= fn.col('FECHA_ADQUISICION'))
    .group_by('EMAIL')
    .agg(
        fn.sum('VENTA').alias('VENTA'),
        fn.count_distinct('ORDER_ID').alias('TRANSACCIONES')
    )
    .with_column('COHORT',
        fn.when(fn.col('TRANSACCIONES') > 3, 4)
        .otherwise(fn.col('TRANSACCIONES'))
    )
    .group_by('COHORT')
    .agg(
        fn.count_distinct('EMAIL').alias('USUARIOS'),
        fn.sum('VENTA').alias('VENTA'),
        fn.sum('TRANSACCIONES').alias('TRANSACCIONES')
    )
    .with_column('TICKET', fn.col('VENTA') / fn.col('TRANSACCIONES'))
    .with_column('FRECUENCIA', fn.col('TRANSACCIONES') / fn.col('USUARIOS'))
)

datos_recompras_semana_DS199O_pd = datos_recompras_semana_DS199O.to_pandas()
datos_recompras_semana_DS199O_pd.to_csv(f'./entregas_semanales_dp/{date.today()}datos_recompras_DS199O_{ANIO_ACTUAL}W{SEMANA_ACTUAL}.csv')
datos_recompras_semana_DS199O_pd

,COHORT,USUARIOS,VENTA,TRANSACCIONES,TICKET,FRECUENCIA
0,1,19742,4.434188e+06,19742,224.606805,1.000000
1,4,8,1.138879e+04,35,325.394089,4.375000
2,2,1216,7.494157e+05,2432,308.147899,2.000000
3,3,107,1.405181e+05,321,437.751101,3.000000


## Datos Recuperacion general

In [20]:
from DP_transacciones_total import get_transacciones_dp_total
from general_activos_en_serie_de_tiempo import get_activos_en_serie_de_tiempo
from general_primeras_compras import get_primeras_compras
from general_get_tiempo import get_tiempo

transacciones_dp_total = get_transacciones_dp_total(session)

tiempo = get_tiempo(session)

primeras_compras = (
    get_primeras_compras(transacciones_dp_total)
    .join(tiempo, on = 'FECHA')
    .select('EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA')
    .with_column('PRIMERA_COMPRA', fn.lit(True))
)

activos_en_semana_actual = (
    get_activos_en_serie_de_tiempo(session, transacciones_dp_total, ['ANIO_ALSEA', 'SEM_ALSEA'])
    .select('EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA')
    .distinct()
    .with_column('ACTIVO_EN_SEMANA_ACTUAL', fn.lit(True))
)

activos_en_semana_previa = (
    activos_en_semana_actual
    .with_column('ANIO_ALSEA', fn.when(fn.col('SEM_ALSEA') == 52, fn.col('ANIO_ALSEA') + 1).otherwise(fn.col('ANIO_ALSEA')))
    .with_column('SEM_ALSEA', fn.when(fn.col('SEM_ALSEA') == 52, 1).otherwise(fn.col('SEM_ALSEA') + 1))
    .select('EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA')
    .with_column('ACTIVO_EN_SEMANA_PREVIA', fn.lit(True))
)

movimientos_base = (
    activos_en_semana_actual
    .join(activos_en_semana_previa, on = ['EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA'], how = 'fullouter')
    .join(primeras_compras, on = ['EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA'], how = 'left')
    .with_column(
        'TYPE',
        fn.when(fn.col('PRIMERA_COMPRA'), 'PRIMERAS_COMPRAS') 
        .when(fn.col('ACTIVO_EN_SEMANA_ACTUAL') & fn.col('ACTIVO_EN_SEMANA_PREVIA'), 'ACTIVO_EN_AMBAS_SEMANAS')
        .when(fn.col('ACTIVO_EN_SEMANA_ACTUAL'), 'RECUPERADOS')
        .when(fn.col('ACTIVO_EN_SEMANA_PREVIA'), 'RECIEN_INACTIVADOS')
    )
    .filter(fn.col('TYPE') != 'ACTIVOS_EN_AMBAS_SEMANAS')
)

tiempo = get_tiempo(session)

primera_compra_por_semana = (
    get_transacciones_dp_total(session)
    .join(tiempo, on = 'FECHA')
    .group_by('EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA')
    .agg(fn.min('FECHA').alias('PRIMERA_COMPRA_EN_SEMANA'))
)

recuperados_semanal = (
    movimientos_base
    .filter(fn.col('ANIO_ALSEA') == ANIO_ACTUAL)
    .filter(fn.col('SEM_ALSEA') == SEMANA_ACTUAL)
    .filter(fn.col('TYPE') == 'RECUPERADOS')
    .join(primera_compra_por_semana, on = ['EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA'])
    .with_column_renamed('PRIMERA_COMPRA_EN_SEMANA', 'FECHA_RECUPERACION')
)


datos_recompras_semanal = (
    recuperados_semanal
    .join(transacciones_dp_total, on = 'EMAIL')
    .filter(fn.col('FECHA') >= fn.col('FECHA_RECUPERACION'))
    .group_by('EMAIL')
    .agg(
        fn.sum('VENTA').alias('VENTA'),
        fn.count_distinct('ORDER_ID').alias('TRANSACCIONES')
    )
    .order_by('TRANSACCIONES', ascending = False)
    .with_column('COHORT',
        fn.when(fn.col('TRANSACCIONES') > 3, 4)
        .otherwise(fn.col('TRANSACCIONES'))
    )
    .group_by('COHORT')
    .agg(
        fn.count_distinct('EMAIL').alias('USUARIOS'),
        fn.sum('VENTA').alias('VENTA'),
        fn.sum('TRANSACCIONES').alias('TRANSACCIONES')
    )
    .with_column('TICKET', fn.col('VENTA') / fn.col('TRANSACCIONES'))
    .with_column('FRECUENCIA', fn.col('TRANSACCIONES') / fn.col('USUARIOS'))
)

datos_recompras_semanal_pd = datos_recompras_semanal.to_pandas()
datos_recompras_semanal_pd.to_csv(f'./entregas_semanales_dp/{date.today()}datos_recompras_recuperacion_{ANIO_ACTUAL}W{SEMANA_ACTUAL}.csv')
datos_recompras_semanal_pd

,COHORT,USUARIOS,VENTA,TRANSACCIONES,TICKET,FRECUENCIA
0,1,26626,6.463829e+06,26626,242.763785,1.000000
1,4,17,1.862241e+04,70,266.034483,4.117647
2,3,97,6.375603e+04,291,219.092902,3.000000
3,2,1305,5.934236e+05,2610,227.365372,2.000000


In [21]:
from DP_transacciones_total import get_transacciones_olo, get_transacciones_cloud

descuentos_DS199O = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3')
    .filter(fn.col('FCIDDESCUENTO').isin([4625, 4623, 4626]))
    .select(['FCIDTIENDA', 'FITICKET'])
)

ordenes_DS199O = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFORDENES')
    .filter(fn.col('FIIDCLIENTE').is_not_null())
    .filter(fn.col('FIIDCLIENTE') != 0)
    .with_column('ID_CLIENTE', fn.concat(fn.col('FCIDTIENDA'), fn.col('FIIDCLIENTE')))
    .with_column('FECHA', fn.to_date('FDIDDIA'))
    .join(descuentos_DS199O, on = ['FCIDTIENDA', 'FITICKET'], how = 'INNER')
)

clientes_DS199O = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACCLIENTES')
    .with_column('ID_CLIENTE', fn.concat(fn.col('FIIDTIENDA'), fn.col('FIIDCLIENTE')))
    .join(ordenes_DS199O, on = 'ID_CLIENTE')
)

transacciones_olo = get_transacciones_olo(session)

redenciones_olo_DS199O = (
    transacciones_olo
    .join(clientes_DS199O, on = ['ID_CLIENTE', 'FECHA'])
    .select(['EMAIL', 'FECHA'])
)

redenciones_cloud_DS199O = (
    get_transacciones_cloud(session)
    .filter(fn.col('COUPONSCODE').like('%DS199O%'))
    .select(['EMAIL', 'FECHA'])
)

redenciones_totales_DS199O = redenciones_cloud_DS199O.union_all(redenciones_olo_DS199O)

redenciones_octubre_DS199O = (
    redenciones_totales_DS199O
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == ANIO_ACTUAL)
    .filter(fn.col('SEM_ALSEA') == SEMANA_ACTUAL)
    .with_column_renamed('FECHA', 'FECHA_RECUPERACION')
    .select('EMAIL', 'FECHA_RECUPERACION')
)

redenciones_octubre_DS199O.agg(fn.count_distinct('EMAIL')).show()

recuperados_semanal = (
    movimientos_base
    .filter(fn.col('ANIO_ALSEA') == ANIO_ACTUAL)
    .filter(fn.col('SEM_ALSEA') == SEMANA_ACTUAL)
    .filter(fn.col('TYPE') == 'RECUPERADOS')
    .join(primera_compra_por_semana, on = ['EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA'])
    .join(redenciones_octubre_DS199O, on = 'EMAIL')
    # .with_column_renamed('PRIMERA_COMPRA_EN_SEMANA', 'FECHA_RECUPERACION')
)

datos_recompras_octubre_DS199O = (
    recuperados_semanal
    .join(transacciones_dp_total, on = 'EMAIL')
    .filter(fn.col('FECHA') >= fn.col('FECHA_RECUPERACION'))
    .group_by('EMAIL')
    .agg(
        fn.sum('VENTA').alias('VENTA'),
        fn.count_distinct('ORDER_ID').alias('TRANSACCIONES')
    )
    .order_by('TRANSACCIONES', ascending = False)
    .with_column('COHORT',
        fn.when(fn.col('TRANSACCIONES') > 3, 4)
        .otherwise(fn.col('TRANSACCIONES'))
    )
    .group_by('COHORT')
    .agg(
        fn.count_distinct('EMAIL').alias('USUARIOS'),
        fn.sum('VENTA').alias('VENTA'),
        fn.sum('TRANSACCIONES').alias('TRANSACCIONES')
    )
    .with_column('TICKET', fn.col('VENTA') / fn.col('TRANSACCIONES'))
    .with_column('FRECUENCIA', fn.col('TRANSACCIONES') / fn.col('USUARIOS'))
)

datos_recompras_octubre_DS199O_pd = datos_recompras_octubre_DS199O.to_pandas()
datos_recompras_octubre_DS199O_pd.to_csv(f'./entregas_semanales_dp/{date.today()}datos_recompras_recuperacion_DS199O_{ANIO_ACTUAL}W{SEMANA_ACTUAL}.csv')
datos_recompras_octubre_DS199O_pd

---------------------------
|"COUNT(DISTINCT EMAIL)"  |
---------------------------
|106871                   |
---------------------------



,COHORT,USUARIOS,VENTA,TRANSACCIONES,TICKET,FRECUENCIA
0,1,15100,3.487293e+06,15100,230.946529,1.000000
1,4,15,4.853103e+04,62,782.758621,4.133333
2,3,63,9.315431e+04,189,492.879949,3.000000
3,2,806,5.915957e+05,1612,366.994845,2.000000


In [22]:
datos_recompras_octubre_DS199O_pd.to_csv(f'./entregas_semanales_dp/{date.today()}datos_recompras_recuperacion_DS199O_{ANIO_ACTUAL}W{SEMANA_ACTUAL}.csv')